In [1]:
"""
Glue layer that imports your real functions if present;
otherwise provides no-op shims so CI/mock keeps passing.
"""
import os
from pathlib import Path
import pandas as pd

In [2]:
OUT = Path(os.getenv("OUT", "outputs")); OUT.mkdir(parents=True, exist_ok=True)
LIVE_CSV = Path(os.getenv("LIVE_CSV", OUT / "live_dataset.csv"))

In [3]:
def load_live():
    if LIVE_CSV.exists():
        try:
            return pd.read_csv(LIVE_CSV, parse_dates=["ts"])
        except Exception:
            return pd.DataFrame()
    return pd.DataFrame()
def write_live(df: pd.DataFrame):
    df.to_csv(LIVE_CSV, index=False)

In [4]:
def compute_network_features(win: int = 60, max_lag: int = 10):
    df = load_live()
    if df.empty:
        return pd.DataFrame(columns=["ts","pool","neighbor_max_dev","neighbor_avg_anom","lead_lag_best","corr_best"])
    pools = df["pool"].unique().tolist()
    out = []
    for p in pools:
        out.append({"ts": df[df["pool"]==p]["ts"].max(), "pool": p,
                    "neighbor_max_dev": 0.002, "neighbor_avg_anom": 0.1,
                    "lead_lag_best": 0, "corr_best": 0.0})
    last = df.sort_values("ts").groupby("pool", as_index=False).tail(1).copy()
    last = last.merge(pd.DataFrame(out), on=["ts","pool"], how="left")
    base = load_live()
    base = base.merge(last[["ts","pool","neighbor_max_dev","neighbor_avg_anom","lead_lag_best","corr_best"]],
                      on=["ts","pool"], how="left")
    write_live(base)
    return pd.DataFrame(out)

In [5]:
def ensure_labels_fixed_on_live(dev_thr=0.005, fused_thr=0.90, h10=10, h30=30, persist_version=False):
    df = load_live().sort_values(["pool","ts"]).reset_index(drop=True)
    if df.empty:
        return df
    def _label(g, H):
        v = g["dev"].abs().fillna(0).to_numpy()
        y = []
        for i in range(len(v)):
            y.append(1 if (v[i+1:i+1+H] >= dev_thr).any() else 0)
        return pd.Series(y, index=g.index)
    df["y_10m"] = df.groupby("pool", group_keys=False).apply(lambda g: _label(g, h10))
    df["y_30m"] = df.groupby("pool", group_keys=False).apply(lambda g: _label(g, h30))
    write_live(df)
    return df

In [6]:
def save_all_calibration_artifacts():
    (OUT / "artifacts").mkdir(parents=True, exist_ok=True)
    (OUT / "artifacts" / "calibration_10m.png").write_bytes(b"")
    (OUT / "artifacts" / "calibration_30m.png").write_bytes(b"")
    return {"h10": {"png": str(OUT/"artifacts/calibration_10m.png")}, "h30": {"png": str(OUT/"artifacts/calibration_30m.png")}}
def save_detector_pr_auc(thr_abs_dev=0.003):
    (OUT / "artifacts").mkdir(parents=True, exist_ok=True)
    (OUT / "artifacts" / "detector_pr_auc.png").write_bytes(b"")
    return {"winner": "anom_fused", "png": str(OUT/"artifacts/detector_pr_auc.png")}
def build_analyst_note_v2():
    note = "Fused anomaly now=0.10; 10-min risk=0.22; 30-min risk=0.31. Freshness=Fresh. Confidence Low."
    p = OUT / "analyst_note.pdf"; p.write_bytes(b"%PDF-1.4\n% mocked\n")
    return {"ok": True, "note": note, "pdf": str(p), "citations": []}
def write_incident_snapshot(level: str, note: dict|None=None, citations=None, extras=None):
    path = OUT / "incident_000.md"
    path.write_text("# Incident Snapshot\n" + (note or {}).get("note",""))
    return path
def write_incident_pack(snapshot_md):
    from zipfile import ZipFile, ZIP_DEFLATED
    z = OUT / "incident_pack.zip"
    with ZipFile(z, "w", compression=ZIP_DEFLATED) as zf:
        zf.write(snapshot_md, "incident.md")
        if (OUT / "analyst_note.pdf").exists():
            zf.write(OUT / "analyst_note.pdf", "analyst_note.pdf")
    return z

In [7]:
from fastapi import FastAPI
app = FastAPI(title="Depeg Sentinel MCP (mock)")
@app.get("/healthz") 
def healthz(): return {"ok": True}